# Reto de vida

En este python notebook esta por separado todo lo relacionado a prueba de vida y comprobación de identidad

In [1]:
pip install opencv-python mediapipe


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
pip uninstall opencv-python

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Prueba de versión de opencv
import cv2
print("OpenCV:", cv2.__version__)
print("Built with GUI support:", hasattr(cv2, 'imshow'))

OpenCV: 4.11.0
Built with GUI support: True


In [ ]:
# Verificar que la camara esta funcionando correctamente
import cv2

cap = cv2.VideoCapture(1)
if cap.isOpened():
    print("✅ Cámara detectada correctamente.")
else:
    print("❌ No se detectó la cámara. Prueba con otro índice.")
cap.release()


✅ Cámara detectada correctamente.


In [19]:
import torch
print("¿GPU disponible?:", torch.cuda.is_available())
print("GPU actual:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Ninguna")


¿GPU disponible?: True
GPU actual: NVIDIA GeForce RTX 4070 Ti


# Prueba de vida

El siguiente código consiste en que el usuario haga tres papadeos, dos giros diciendo que si y dos giros diciendo que no

In [17]:
import cv2
import mediapipe as mp
import time

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

LEFT_EYE_FULL = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_FULL = [362, 385, 387, 263, 373, 380]

def eye_aspect_ratio(landmarks, eye_indices, image_shape):
    def dist(p1, p2):
        x1, y1 = int(p1.x * image_shape[1]), int(p1.y * image_shape[0])
        x2, y2 = int(p2.x * image_shape[1]), int(p2.y * image_shape[0])
        return ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
    A = dist(landmarks[eye_indices[1]], landmarks[eye_indices[5]])
    B = dist(landmarks[eye_indices[2]], landmarks[eye_indices[4]])
    D = dist(landmarks[eye_indices[0]], landmarks[eye_indices[3]])
    return (A + B) / (2.0 * D)

cap = cv2.VideoCapture(0)

print("🟡 Reto de vida: 3 parpadeos (3s), 2 'sí' (5s) y 2 'no' (5s)")

# Parpadeo
blink_counter = 0
blink_ready = True
blink_start_time = None
blink_completed = False

# Movimiento vertical ("sí")
nod_count = 0
nod_stage = "neutral"
nod_start_y = None
nod_start_time = None
nod_completed = False

# Movimiento horizontal ("no")
shake_count = 0
shake_stage = "neutral"
shake_start_x = None
shake_start_time = None
shake_completed = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

            # === Parpadeo ===
            ear_left = eye_aspect_ratio(face_landmarks.landmark, LEFT_EYE_FULL, frame.shape)
            ear_right = eye_aspect_ratio(face_landmarks.landmark, RIGHT_EYE_FULL, frame.shape)
            ear_avg = (ear_left + ear_right) / 2.0
            cv2.putText(frame, f"EAR: {ear_avg:.2f}", (30, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            if not blink_completed:
                if ear_avg < 0.26 and blink_ready:
                    if blink_counter == 0:
                        blink_start_time = time.time()
                    blink_counter += 1
                    blink_ready = False
                    print(f"✅ Parpadeo #{blink_counter} detectado")
                elif ear_avg >= 0.30:
                    blink_ready = True

                if blink_counter > 0 and (time.time() - blink_start_time > 3):
                    print("⏱️ Tiempo excedido. Reiniciando contador de parpadeos.")
                    blink_counter = 0
                    blink_start_time = None

                if blink_counter >= 3 and (time.time() - blink_start_time <= 3):
                    blink_completed = True
                    print("✅ Reto de parpadeos completado")

            # === Movimiento vertical ("sí") ===
            nose_y = face_landmarks.landmark[1].y
            if nod_start_y is None:
                nod_start_y = nose_y
                nod_start_time = time.time()
            delta_y = nose_y - nod_start_y

            if not nod_completed:
                if nod_stage == "neutral" and delta_y > 0.03:
                    nod_stage = "down"
                elif nod_stage == "down" and delta_y < -0.03:
                    nod_stage = "up"
                    nod_count += 1
                    print(f"✅ Asentimiento #{nod_count} detectado")
                    nod_stage = "neutral"

                if time.time() - nod_start_time > 5 and nod_count < 2:
                    print("⏱️ Tiempo para 'sí' agotado. Reiniciando.")
                    nod_count = 0
                    nod_start_time = time.time()

                if nod_count >= 2:
                    nod_completed = True
                    print("✅ Reto de movimiento 'sí' completado")

            # === Movimiento horizontal ("no") ===
            nose_x = face_landmarks.landmark[1].x
            if shake_start_x is None:
                shake_start_x = nose_x
                shake_start_time = time.time()
            delta_x = nose_x - shake_start_x

            if not shake_completed:
                if shake_stage == "neutral" and delta_x > 0.03:
                    shake_stage = "right"
                elif shake_stage == "right" and delta_x < -0.03:
                    shake_stage = "left"
                    shake_count += 1
                    print(f"✅ Negación #{shake_count} detectada")
                    shake_stage = "neutral"

                if time.time() - shake_start_time > 5 and shake_count < 2:
                    print("⏱️ Tiempo para 'no' agotado. Reiniciando.")
                    shake_count = 0
                    shake_start_time = time.time()

                if shake_count >= 2:
                    shake_completed = True
                    print("✅ Reto de movimiento 'no' completado")

    # Mostrar progreso
    cv2.putText(frame, f"Parpadeos: {blink_counter}/3", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, f"Asentir (sí): {nod_count}/2", (30, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.putText(frame, f"Negar (no): {shake_count}/2", (30, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

    cv2.imshow("Reto de Vida", frame)

    if blink_completed and nod_completed and shake_completed:
        break
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

# Resultado final
if blink_completed and nod_completed and shake_completed:
    print("🎉 Reto de vida superado con éxito.")
else:
    print("❌ No se completaron todos los retos de vida.")


🟡 Reto de vida: 3 parpadeos (3s), 2 'sí' (5s) y 2 'no' (5s)
✅ Negación #1 detectada
✅ Negación #2 detectada
✅ Reto de movimiento 'no' completado
⏱️ Tiempo para 'sí' agotado. Reiniciando.
✅ Parpadeo #1 detectado
✅ Parpadeo #2 detectado
⏱️ Tiempo excedido. Reiniciando contador de parpadeos.
✅ Parpadeo #1 detectado
⏱️ Tiempo para 'sí' agotado. Reiniciando.
✅ Parpadeo #2 detectado
✅ Parpadeo #3 detectado
✅ Reto de parpadeos completado
✅ Asentimiento #1 detectado
✅ Asentimiento #2 detectado
✅ Reto de movimiento 'sí' completado
🎉 Reto de vida superado con éxito.


In [12]:
import cv2
import mediapipe as mp
import time

# Inicialización
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Landmarks para ojos
LEFT_EYE_FULL = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_FULL = [362, 385, 387, 263, 373, 380]

# EAR con 6 puntos por ojo
def eye_aspect_ratio(landmarks, eye_indices, image_shape):
    def dist(p1, p2):
        x1, y1 = int(p1.x * image_shape[1]), int(p1.y * image_shape[0])
        x2, y2 = int(p2.x * image_shape[1]), int(p2.y * image_shape[0])
        return ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5

    A = dist(landmarks[eye_indices[1]], landmarks[eye_indices[5]])
    B = dist(landmarks[eye_indices[2]], landmarks[eye_indices[4]])
    D = dist(landmarks[eye_indices[0]], landmarks[eye_indices[3]])
    return (A + B) / (2.0 * D)

# Verifica si el usuario está mirando al frente
def is_looking_forward(landmarks):
    def centered(pupil, outer, inner):
        d1 = abs(pupil - outer)
        d2 = abs(inner - pupil)
        ratio = d1 / (d1 + d2 + 1e-6)
        return 0.425 < ratio < 0.575  # Pupila al centro del ojo

    # Ojo izquierdo
    left_pupil = landmarks[468].x
    left_outer = landmarks[33].x
    left_inner = landmarks[133].x
    left_centered = centered(left_pupil, left_outer, left_inner)

    # Ojo derecho
    right_pupil = landmarks[473].x
    right_outer = landmarks[362].x
    right_inner = landmarks[263].x
    right_centered = centered(right_pupil, right_outer, right_inner)

    return left_centered and right_centered

# Captura de selfie con validación de condiciones
cap = cv2.VideoCapture(0)
print("📸 Colócate en el centro, con los ojos abiertos, mirando a la cámara...")

stable_frame = None
stable_counter = 0
selfie_taken = False
gaze_start_time = None

while cap.isOpened() and not selfie_taken:
    ret, frame = cap.read()

    if not ret:
        break

    
    raw_frame = frame.copy()  # ← copia limpia sin textos ni overlays
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    
    condition_text = []
    selfie_ready = False

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        nose_x = face_landmarks.landmark[1].x
        nose_y = face_landmarks.landmark[1].y

        # 1. Cara centrada
        centered_x = 0.4 < nose_x < 0.6
        centered_y = 0.4 < nose_y < 0.6
        centered = centered_x and centered_y
        condition_text.append("✅ Cara centrada" if centered else "❌ No centrado")

        # 2. Ojos abiertos
        ear_left = eye_aspect_ratio(face_landmarks.landmark, LEFT_EYE_FULL, frame.shape)
        ear_right = eye_aspect_ratio(face_landmarks.landmark, RIGHT_EYE_FULL, frame.shape)
        eyes_open = ear_left > 0.26 and ear_right > 0.26
        condition_text.append("✅ Ojos abiertos" if eyes_open else "❌ Ojos cerrados")

        # 3. Cámara estable
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        if stable_frame is None:
            stable_frame = gray
            continue
        diff = cv2.absdiff(stable_frame, gray)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
        movement = cv2.countNonZero(thresh)
        stable = movement < 5000
        if stable:
            stable_counter += 1
        else:
            stable_counter = 0
        condition_text.append("✅ Cámara estable" if stable_counter >= 5 else "❌ Cámara en movimiento")

        # 4. Mirando a la cámara (mejorado)
        looking_forward = is_looking_forward(face_landmarks.landmark)
        if looking_forward:
            if gaze_start_time is None:
                gaze_start_time = time.time()
            gaze_duration = time.time() - gaze_start_time
        else:
            gaze_start_time = None
            gaze_duration = 0

        if gaze_duration >= 3:
            condition_text.append("✅ Mirando a la cámara")
        else:
            condition_text.append(f"❌ Mirada desviada ({gaze_duration:.1f}s)")

        # Mostrar condiciones en pantalla
        for i, text in enumerate(condition_text):
            cv2.putText(frame, text, (30, 40 + i*30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # Si se cumplen todas las condiciones...
        if centered and eyes_open and stable_counter >= 5 and gaze_duration >= 3:
            cv2.imwrite("selfie.jpg", raw_frame)
            print("📷 Selfie tomada exitosamente y guardada como 'selfie.jpg'")
            selfie_taken = True
            break

        stable_frame = gray

    cv2.imshow("Captura de Selfie", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


📸 Colócate en el centro, con los ojos abiertos, mirando a la cámara...
📷 Selfie tomada exitosamente y guardada como 'selfie.jpg'


In [18]:
import cv2
import pytesseract
import imutils
import time

# Ruta a Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

cap = cv2.VideoCapture(0)
print("🪪 Muestra tu INE al centro de la cámara, bien enfocada y grande...")

stable_counter = 0
required_frames = 150  # ≈5 segundos

while cap.isOpened():
    ret, raw_frame = cap.read()  # raw_frame será el que guardaremos limpio
    if not ret:
        break

    frame = raw_frame.copy()  # este lo usamos para mostrar con texto

    # Detección de bordes
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 75, 200)

    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

    found_card = False
    frame_area = frame.shape[0] * frame.shape[1]

    for c in cnts:
        approx = cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)
        area = cv2.contourArea(c)

        if len(approx) == 4 and area > 0.20 * frame_area:
            cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
            stable_counter += 1
            found_card = True
            break

    if not found_card:
        stable_counter = 0

    # Indicadores visuales SOLO para pantalla (no se guardan en raw_frame)
    porcentaje = int((stable_counter / required_frames) * 100)
    cv2.putText(frame, f"INE detectada: {porcentaje}%", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, "Mantén la INE visible y grande durante 5 segundos",
                (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    cv2.imshow("Detección de INE", frame)

    # Guardar la imagen original SIN textos dibujados
    if stable_counter >= required_frames:
        cv2.imwrite("ine.jpg", raw_frame)
        print("📸 INE capturada sin texto y guardada como 'ine.jpg'")
        break

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


🪪 Muestra tu INE al centro de la cámara, bien enfocada y grande...
📸 INE capturada sin texto y guardada como 'ine.jpg'


In [ ]:
import cv2
import pytesseract
import numpy as np
import os

# Ruta a Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Hermanos\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# Cargar imagen
image_path = "ine.jpg"
if not os.path.exists(image_path):
    print("❌ No se encontró 'ine.jpg'")
    exit()

image = cv2.imread(image_path)
orig = image.copy()

# Convertir a escala de grises y detectar bordes
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blur, 75, 200)

# Buscar contornos
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

doc_cnt = None

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    if len(approx) == 4:
        doc_cnt = approx
        break

if doc_cnt is None:
    print("❌ No se encontró un contorno de INE con 4 lados.")
    exit()

# Reordenar puntos (top-left, top-right, bottom-right, bottom-left)
def reorder_points(pts):
    pts = pts.reshape((4, 2))
    new_pts = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)

    new_pts[0] = pts[np.argmin(s)]
    new_pts[2] = pts[np.argmax(s)]
    new_pts[1] = pts[np.argmin(diff)]
    new_pts[3] = pts[np.argmax(diff)]

    return new_pts

# Aplicar transformación de perspectiva (warp)
pts = reorder_points(doc_cnt)
(tl, tr, br, bl) = pts

widthA = np.linalg.norm(br - bl)
widthB = np.linalg.norm(tr - tl)
heightA = np.linalg.norm(tr - br)
heightB = np.linalg.norm(tl - bl)
maxWidth = max(int(widthA), int(widthB))
maxHeight = max(int(heightA), int(heightB))

dst = np.array([
    [0, 0],
    [maxWidth - 1, 0],
    [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]
], dtype="float32")

M = cv2.getPerspectiveTransform(pts, dst)
warped = cv2.warpPerspective(orig, M, (maxWidth, maxHeight))

# Preprocesamiento del warp para OCR
warp_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
warp_gray = cv2.threshold(warp_gray, 120, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# OCR sobre la imagen corregida
custom_config = r'--oem 3 --psm 6 -l spa'
text = pytesseract.image_to_string(warp_gray, config=custom_config)

# Resultado
print("📝 Texto extraído de la INE (con corrección de perspectiva):")
print("------------------------------------------------------------")
print(text)
print("------------------------------------------------------------")

# Guardar la imagen corregida (opcional)
cv2.imwrite("ine_rectificada.jpg", warped)


Resultado:
```
📝 Texto extraído de la INE (con corrección de perspectiva):
------------------------------------------------------------
3 uo AX
_ NOMBRE DH
= FLORES
MENDOZA
a JÓSUE EMMAMEL
: lors,
Point
CAAWEDEELECTOR FLMNIB09031919H600.
A E

------------------------------------------------------------

In [1]:
pip install easyocr


  Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
  Using cached ninja-1.11.1.4-py3-none-win_amd64.whl (296 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import easyocr
import cv2

reader = easyocr.Reader(['es'])  # Español

image_path = 'ine.jpg'
image = cv2.imread(image_path)

results = reader.readtext(image)

print("📝 Texto detectado:")
for bbox, text, confidence in results:
    print(f"- {text} ({confidence:.2f})")



Resultado:
```
Downloading detection model, please wait. This may take several minutes depending upon your network connection.
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Downloading recognition model, please wait. This may take several minutes depending upon your network connection.
Progress: |██████████████████████████████████████████████████| 100.0% Complete📝 Texto detectado:
- INSTITUTO NACIONAL ELECTORAL (0.99)
- MÉXICO   CREDENCIAL PARA VoTAR (0.57)
- SEXOH (0.64)
- NOMBRE (0.84)
- FLORES (1.00)
- MENDOZA (0.99)
- JOSUEEMMANUEL (0.98)
- DOMICLIO (0.53)
- ************ (0.90)
- ******** (0.76)
- ************** (0.74)
- ********** (0.60)
- CURP (0.91)
- AÑO DEREGISTRO (0.78)
- ********** (0.68)
- ******** (0.80)
- FECHA DENACIMIENTO (0.73)
- SECCION (0.78)
- VIGENCIA (0.72)
- ********** (0.71)
- ****** (1.00)
- ********** (0.86)

In [6]:
pip install paddleocr

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [13]:
pip install paddlepaddle==3.0.0 -i https://www.paddlepaddle.org.cn/packages/stable/cpu/

Looking in indexes: https://www.paddlepaddle.org.cn/packages/stable/cpu/
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='es')
print("✅ PaddleOCR funcionando (aunque use CPU, no se chocará con PyTorch)")

c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to C:\Users\Hermanos/.paddleocr/whl\det\en\en_PP-OCRv3_det_infer\en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:16<00:00, 241.16it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to C:\Users\Hermanos/.paddleocr/whl\rec\latin\latin_PP-OCRv3_rec_infer\latin_PP-OCRv3_rec_infer.tar


100%|██████████| 9930/9930 [00:18<00:00, 541.70it/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\Hermanos/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:16<00:00, 128.67it/s]

[2025/05/07 14:06:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='

✅ PaddleOCR funcionando (aunque use CPU, no se chocará con PyTorch)


In [20]:
from paddleocr import PaddleOCR
import cv2

# Inicializar PaddleOCR con idioma español y CPU
ocr = PaddleOCR(use_angle_cls=True, lang='es')  # use_angle_cls corrige texto inclinado

# Ruta de la imagen
image_path = 'ine.jpg'

# Ejecutar OCR
results = ocr.ocr(image_path, cls=True)

# Mostrar resultados
print("📝 Texto detectado en la imagen:")
print("----------------------------------------------------")
for line in results[0]:  # results[0] contiene la lista de líneas detectadas
    bbox, (text, confidence) = line
    print(f"- {text} (confianza: {confidence:.2f})")
print("----------------------------------------------------")


[2025/05/07 20:08:09] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='


Resultado:
```
[2025/05/07 20:08:09] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='c:\\Users\\Hermanos\\Desktop\\Proyecto Deepfake\\.venv-mediapipe\\lib\\site-packages\\paddleocr\\ppocr\\utils\\dict\\latin_dict.txt', use_space_char=True, vis_font_path='./doc/fonts/simfang.ttf', drop_score=0.5, e2e_algorithm='PGNet', e2e_model_dir=None, e2e_limit_side_len=768, e2e_limit_type='max', e2e_pgnet_score_thresh=0.5, e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_pgnet_valid_set='totaltext', e2e_pgnet_mode='fast', use_angle_cls=True, cls_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_image_shape='3, 48, 192', label_list=['0', '180'], cls_batch_num=6, cls_thresh=0.9, enable_mkldnn=False, cpu_threads=10, use_pdserving=False, warmup=False, sr_model_dir=None, sr_image_shape='3, 32, 128', sr_batch_num=1, draw_img_save_dir='./inference_results', save_crop_res=False, crop_res_save_dir='./output', use_mp=False, total_process_num=1, process_id=0, benchmark=False, save_log_path='./log_output/', show_log=True, use_onnx=False, onnx_providers=False, onnx_sess_options=False, return_word_box=False, output='./output', table_max_len=488, table_algorithm='TableAttn', table_model_dir=None, merge_no_span_structure=True, table_char_dict_path=None, formula_algorithm='LaTeXOCR', formula_model_dir=None, formula_char_dict_path=None, formula_batch_num=1, layout_model_dir=None, layout_dict_path=None, layout_score_threshold=0.5, layout_nms_threshold=0.5, kie_algorithm='LayoutXLM', ser_model_dir=None, re_model_dir=None, use_visual_backbone=True, ser_dict_path='../train_data/XFUND/class_list_xfun.txt', ocr_order_method=None, mode='structure', image_orientation=False, layout=True, table=True, formula=False, ocr=True, recovery=False, recovery_to_markdown=False, use_pdf2docx_api=False, invert=False, binarize=False, alphacolor=(255, 255, 255), lang='es', det=True, rec=True, type='ocr', savefile=False, ocr_version='PP-OCRv4', structure_version='PP-StructureV2')
[2025/05/07 20:08:10] ppocr DEBUG: dt_boxes num : 22, elapsed : 0.04807305335998535
[2025/05/07 20:08:10] ppocr DEBUG: cls num  : 22, elapsed : 0.06923294067382812
[2025/05/07 20:08:10] ppocr DEBUG: rec_res num  : 22, elapsed : 0.24478578567504883
📝 Texto detectado en la imagen:
----------------------------------------------------
- MEXICO (confianza: 0.99)
- INSTITUTO NACIONAL ELECTORAL (confianza: 0.96)
- CREDENCIAL PARAVOTAR (confianza: 0.98)
- FLORES (confianza: 1.00)
- NOMERE (confianza: 0.93)
- SEXOH (confianza: 0.98)
- JOSUEEMMANUEL (confianza: 0.97)
- ******** (confianza: 0.96)
- DOMICILIO (confianza: 0.94)
- ****** (confianza: 0.96)
- ********** (confianza: 0.94)
- ************ (confianza: 0.93)
- ******** (confianza: 0.97)
- CURP (confianza: 0.99)
- ANODEREGISTRO (confianza: 0.88)
- ******** (confianza: 0.99)
- FECHA DE NACIMENTO (confianza: 0.87)
- ******** (confianza: 0.92)
- SOIOOON (confianza: 0.53)
- VIGENCIA (confianza: 0.99)
- ****** (confianza: 0.99)
- ****** (confianza: 0.99)
----------------------------------------------------

In [ ]:
from paddleocr import PaddleOCR
import re

# Inicializa OCR
ocr = PaddleOCR(use_angle_cls=True, lang='es')
results = ocr.ocr('ine.jpg', cls=True)

# Extraer todas las líneas detectadas por OCR
lineas_ocr_ine = [line[1][0].strip().upper() for line in results[0]]

# Buscar CURP
curp_match = None
for linea in lineas_ocr_ine:
    if re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", linea):
        curp_match = re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", linea)
        break
curp = curp_match.group() if curp_match else "No detectado"

# Buscar fecha de nacimiento
fecha_match = None
for linea in lineas_ocr_ine:
    if re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", linea):
        fecha_match = re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", linea)
        break
fecha = fecha_match.group() if fecha_match else "No detectada"

# Mostrar resultados
print("📌 CURP:", curp)
print("📌 Fecha de nacimiento:", fecha)
print("📌 Líneas OCR de la INE:")
for l in lineas_ocr_ine:
    print("-", l)


Resultado:
```
[2025/05/07 16:04:34] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='c:\\Users\\Hermanos\\Desktop\\Proyecto Deepfake\\.venv-mediapipe\\lib\\site-packages\\paddleocr\\ppocr\\utils\\dict\\latin_dict.txt', use_space_char=True, vis_font_path='./doc/fonts/simfang.ttf', drop_score=0.5, e2e_algorithm='PGNet', e2e_model_dir=None, e2e_limit_side_len=768, e2e_limit_type='max', e2e_pgnet_score_thresh=0.5, e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_pgnet_valid_set='totaltext', e2e_pgnet_mode='fast', use_angle_cls=True, cls_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_image_shape='3, 48, 192', label_list=['0', '180'], cls_batch_num=6, cls_thresh=0.9, enable_mkldnn=False, cpu_threads=10, use_pdserving=False, warmup=False, sr_model_dir=None, sr_image_shape='3, 32, 128', sr_batch_num=1, draw_img_save_dir='./inference_results', save_crop_res=False, crop_res_save_dir='./output', use_mp=False, total_process_num=1, process_id=0, benchmark=False, save_log_path='./log_output/', show_log=True, use_onnx=False, onnx_providers=False, onnx_sess_options=False, return_word_box=False, output='./output', table_max_len=488, table_algorithm='TableAttn', table_model_dir=None, merge_no_span_structure=True, table_char_dict_path=None, formula_algorithm='LaTeXOCR', formula_model_dir=None, formula_char_dict_path=None, formula_batch_num=1, layout_model_dir=None, layout_dict_path=None, layout_score_threshold=0.5, layout_nms_threshold=0.5, kie_algorithm='LayoutXLM', ser_model_dir=None, re_model_dir=None, use_visual_backbone=True, ser_dict_path='../train_data/XFUND/class_list_xfun.txt', ocr_order_method=None, mode='structure', image_orientation=False, layout=True, table=True, formula=False, ocr=True, recovery=False, recovery_to_markdown=False, use_pdf2docx_api=False, invert=False, binarize=False, alphacolor=(255, 255, 255), lang='es', det=True, rec=True, type='ocr', savefile=False, ocr_version='PP-OCRv4', structure_version='PP-StructureV2')
[2025/05/07 16:04:35] ppocr DEBUG: dt_boxes num : 23, elapsed : 0.05400681495666504
[2025/05/07 16:04:35] ppocr DEBUG: cls num  : 23, elapsed : 0.03323245048522949
[2025/05/07 16:04:35] ppocr DEBUG: rec_res num  : 23, elapsed : 0.12092781066894531
📌 CURP: FOMJ030313HNLLNSA2
📌 Fecha de nacimiento: 13/03/2003
📌 Líneas OCR de la INE:
- MEXICO
- INSTITUTO NACIONAL ELECTORAL
- CREDENCIAL PARAVOTAR
- NOMBRE
- SEXOH
- FLORES
- MENDOZA
- JOSUE EMMANUEL
- DOMICILIO
- **********
- ********
- ********
- **********
- CURP
- ANODEREGISTRO
- **********
- ******
- FECHA DE NACIMIENTO
- SECCION
- **********
- ********
- VIGENCIA
- ********

In [6]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

from paddleocr import PaddleOCR
import re

# ------------------------------------
# ✅ Función para validar coincidencias de nombre por partes
# ------------------------------------
def coincide_nombre_web_con_ocr(nombre_web, ap1, ap2, lineas_ocr):
    partes = [nombre_web, ap1, ap2]
    for parte in partes:
        if not any(parte.upper() in linea for linea in lineas_ocr):
            return False
    return True

# Configurar Chrome en modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

# Inicializa OCR
ocr = PaddleOCR(use_angle_cls=True, lang='es')
results = ocr.ocr('ine.jpg', cls=True)

# Extraer todas las líneas detectadas por OCR
lineas_ocr_ine = [line[1][0].strip().upper() for line in results[0]]

# Buscar CURP
curp_match = None
for linea in lineas_ocr_ine:
    if re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", linea):
        curp_match = re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", linea)
        break
curp = curp_match.group() if curp_match else "No detectado"

# Buscar fecha de nacimiento
fecha_match = None
for linea in lineas_ocr_ine:
    if re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", linea):
        fecha_match = re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", linea)
        break
fecha = fecha_match.group() if fecha_match else "No detectada"

# Mostrar resultados
print("📌 CURP:", curp)
print("📌 Fecha de nacimiento:", fecha)
print("📌 Líneas OCR de la INE:")
for l in lineas_ocr_ine:
    print("-", l)

# Iniciar navegador
driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get("https://www.gob.mx/curp/")

    input_curp = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "curpinput"))
    )
    input_curp.send_keys(curp)

    boton_buscar = driver.find_element(By.ID, "searchButton")
    boton_buscar.click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//td[@style='text-transform: uppercase;']"))
    )

    celdas = driver.find_elements(By.XPATH, "//td[@style='text-transform: uppercase;']")
    if len(celdas) >= 7:
        curp_web = celdas[0].text.strip()
        nombre_web = celdas[1].text.strip()
        apellido1_web = celdas[2].text.strip()
        apellido2_web = celdas[3].text.strip()
        fecha_web = celdas[5].text.strip()

        nombre_completo_web = f"{apellido1_web} {apellido2_web} {nombre_web}"

        print("📄 Datos extraídos del sitio:")
        print("CURP:", curp_web)
        print("Nombre completo:", nombre_completo_web)
        print("Fecha de nacimiento:", fecha_web)

        # Validación final
        nombre_ok = coincide_nombre_web_con_ocr(nombre_web, apellido1_web, apellido2_web, lineas_ocr_ine)
        fecha_ok = (fecha.strip() == fecha_web.strip())

        if nombre_ok and fecha_ok:
            print("✅ Identidad confirmada con datos oficiales.")
        else:
            print("❌ No coinciden los datos. Identidad no confirmada.")

    else:
        print("❌ No se encontraron suficientes celdas para extraer los datos esperados.")

finally:
    time.sleep(2)
    driver.quit()


In [22]:
import cv2
import mediapipe as mp

# Inicializar MediaPipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.6)

# Cargar imagen de la INE
image = cv2.imread("ine.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = face_detection.process(image_rgb)

# Detectar cara más grande
if results.detections:
    h, w, _ = image.shape
    max_area = 0
    best_face = None

    for detection in results.detections:
        bboxC = detection.location_data.relative_bounding_box
        x, y = int(bboxC.xmin * w), int(bboxC.ymin * h)
        w_box, h_box = int(bboxC.width * w), int(bboxC.height * h)

        area = w_box * h_box
        if area > max_area:
            max_area = area
            best_face = (x, y, w_box, h_box)

    if best_face:
        x, y, w_box, h_box = best_face
        x, y = max(0, x), max(0, y)
        recorte = image[y:y + h_box, x:x + w_box]
        cv2.imwrite("ine_face.jpg", recorte)
        print("✅ Se guardó la cara más grande como 'ine_face.jpg'")
else:
    print("❌ No se detectaron rostros en la INE.")


✅ Se guardó la cara más grande como 'ine_face.jpg'


In [4]:
pip install face_recognition

  Created wheel for dlib: filename=dlib-19.24.8-cp310-cp310-win_amd64.whl size=2920432 sha256=286b6b180094c13461b6745ca165cf2b9b7bf332d83e33c561a45363e5a4f3e3
  Stored in directory: c:\users\hermanos\appdata\local\pip\cache\wheels\5b\b1\d3\dc280060d84aced767d0f4a6a633e487182ce3a760c61ab200
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566199 sha256=bbfcf9dce5944dbdcd87a23c19253a3573d090f53e246ffa4ca18531359c3507
  Stored in directory: c:\users\hermanos\appdata\local\pip\cache\wheels\7a\eb\cf\e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built dlib face-recognition-models
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [23]:
import face_recognition

# Cargar imágenes
img1 = face_recognition.load_image_file("selfie.jpg")
img2 = face_recognition.load_image_file("ine_face.jpg")

# Obtener codificaciones
enc1 = face_recognition.face_encodings(img1)
enc2 = face_recognition.face_encodings(img2)

if enc1 and enc2:
    result = face_recognition.compare_faces([enc1[0]], enc2[0])
    distance = face_recognition.face_distance([enc1[0]], enc2[0])[0]
    
    if result[0]:
        print(f"✅ Rostros coinciden (distancia: {distance:.4f})")
    else:
        print(f"❌ Rostros NO coinciden (distancia: {distance:.4f})")
else:
    print("❌ No se pudieron codificar ambos rostros correctamente.")


✅ Rostros coinciden (distancia: 0.4490)
